# 2. Options in Optim.jl
This notebook shows how to configure the default options in Optim.jl. Numerical optimization can sometimes require a bit of experimentation to get right in a given context, and this requires the ability to change various details, but also study how the iterations proceed.

## 2.1 Method options
Many of the methods in Optim.jl are developped as, and meant to be, general purpose non-linear optimization procedures, if there ever was such a thing. However, no matter what algorithm you use, there are choices of constants, line searches, and more. These things can have significant influence on the success of locating a minimum, so it's important to know how to do it.

### 2.1.1 Line search choice
Many methods in Optim.jl are build around determining a *search direction*, $s$. This direction is not the final step-size however. To guarantee convergence to a critical point, we use line search algorithms to find a step size $\alpha$, such that the final update
$$
    x_{k+1}=x_k+\alpha\cdot s
$$
To illustrate, let's find a minimizer to the objective function we will call "Large Polynomial".

In [40]:
function f(x)
    return (x[1] + 10.0 * x[2])^2 + 5.0 * (x[3] - x[4])^2 +
            (x[2] - 2.0 * x[3])^4 + 10.0 * (x[1] - x[4])^4
end

function g!(storage, x)
    storage[1] = 2.0 * (x[1] + 10.0 * x[2]) + 40.0 * (x[1] - x[4])^3
    storage[2] = 20.0 * (x[1] + 10.0 * x[2]) + 4.0 * (x[2] - 2.0 * x[3])^3
    storage[3] = 10.0 * (x[3] - x[4]) - 8.0 * (x[2] - 2.0 * x[3])^3
    storage[4] = -10.0 * (x[3] - x[4]) - 40.0 * (x[1] - x[4])^3
end

function h!(storage, x)
    storage[1, 1] = 2.0 + 120.0 * (x[1] - x[4])^2
    storage[1, 2] = 20.0
    storage[1, 3] = 0.0
    storage[1, 4] = -120.0 * (x[1] - x[4])^2
    storage[2, 1] = 20.0
    storage[2, 2] = 200.0 + 12.0 * (x[2] - 2.0 * x[3])^2
    storage[2, 3] = -24.0 * (x[2] - 2.0 * x[3])^2
    storage[2, 4] = 0.0
    storage[3, 1] = 0.0
    storage[3, 2] = -24.0 * (x[2] - 2.0 * x[3])^2
    storage[3, 3] = 10.0 + 48.0 * (x[2] - 2.0 * x[3])^2
    storage[3, 4] = -10.0
    storage[4, 1] = -120.0 * (x[1] - x[4])^2
    storage[4, 2] = 0.0
    storage[4, 3] = -10.0
    storage[4, 4] = 10.0 + 120.0 * (x[1] - x[4])^2
end

h! (generic function with 2 methods)

We will try to optimize this function from a starting point of $x=[3, -1, 0, 1]$, and the result should be the zero vector. We explicitly choose the line search by Hager and Zhang developped for the very Conjugate Gradient implementation we have in Optim.jl.

In [48]:
using Optim
optimize(f, g!, h!, [3.0, -1.0, 0.0, 1.0], ConjugateGradient(linesearch=LineSearches.HagerZhang()))

Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [3.0,-1.0,0.0,1.0]
 * Minimizer: [0.003075918767500598,-0.00030759239566266136, ...]
 * Minimum: 1.864396e-10
 * Iterations: 1000
 * Convergence: false
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.70e-06 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 4.83e-03 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 2.83e-06 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: true
 * Objective Calls: 2002
 * Gradient Calls: 1177

Notice, that the convergence information tells us that we are not at a local minimizer. The procedure stopped because we reached the default value of 1000 iterations.

In [45]:
optimize(f, g!, h!, [3.0, -1.0, 0.0, 1.0], ConjugateGradient(linesearch=LineSearches.MoreThuente()))

Results of Optimization Algorithm
 * Algorithm: Conjugate Gradient
 * Starting Point: [3.0,-1.0,0.0,1.0]
 * Minimizer: [-0.0006377700262345379,6.377700370473088e-5, ...]
 * Minimum: 3.515308e-13
 * Iterations: 469
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.39e-09 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.73e-05 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 9.96e-09 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 948
 * Gradient Calls: 571

With a different line search algorithm by More and Thuente, we get much better performance. Let us see how Newton's method that uses the Hessian information does with the Hager and Zhang line search.

In [50]:
optimize(f, g!, h!, [3.0, -1.0, 0.0, 1.0], Newton(linesearch=LineSearches.HagerZhang()))

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [3.0,-1.0,0.0,1.0]
 * Minimizer: [0.00018829366513309454,-1.8829395155003336e-5, ...]
 * Minimum: 6.297554e-15
 * Iterations: 10
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 9.24e-05 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.73e-09 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 31
 * Gradient Calls: 31
 * Hessian Calls: 10

We see that it does much better, will we see a difference once more when switching to More and Thuente's line search.

In [52]:
optimize(f, g!, h!, [3.0, -1.0, 0.0, 1.0], Newton(linesearch=LineSearches.MoreThuente()))

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [3.0,-1.0,0.0,1.0]
 * Minimizer: [0.0007160206186411141,-7.160206186411139e-5, ...]
 * Minimum: 1.316816e-12
 * Iterations: 20
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.58e-04 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = NaN 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 8.70e-09 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 21
 * Gradient Calls: 21
 * Hessian Calls: 20

## 2.2 General options

While some options are algorithm specific, some are sufficiently general, that they're included in the `Optim.Options` type. Things in this category are: number of iterations, convergence criterions, etc. Below, we see the the different keywords and their default values.

```julia
Options(;
        x_tol::Real = 1e-32,
        f_tol::Real = 1e-32,
        g_tol::Real = 1e-8,
        f_calls_limit::Int = 0,
        g_calls_limit::Int = 0,
        h_calls_limit::Int = 0,
        time_limit = NaN,
        allow_f_increases::Bool = false,
        iterations::Integer = 1_000,
        store_trace::Bool = false,
        show_trace::Bool = false,
        extended_trace::Bool = false,
        show_every::Integer = 1,
        callback = nothing)
```

The first three define convergence tolerances, the next three define soft limits on the number of calls to the objective function, the gradient and the hessian, then there's a time limit (in seconds), an option to allow the objective to increase while iterating, the number of iterations we allow, some trace related options and lastly you can pass a callback.